<a href="https://colab.research.google.com/github/aparnavinayankozhipuram/Mistral-7B-Evaluation/blob/main/27_April_MIT_Tinyllama_RAG_Score_Evaluation_3rd_query_9th_iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk rouge-score
!pip install scikit-learn


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=4f99fd8c16c3a5f6e6a4cce933a18259eea9a86826808c3977c0e9b25259ee1a
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [2]:
import nltk
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Ensure necessary NLTK data is downloaded
# The original line was: nltk.download('punkt')
nltk.download('punkt_tab') # Download punkt_tab instead of just punkt


def evaluate_rag_model(predictions, references):
    """
    Evaluate RAG model using BLEU and ROUGE scores.

    Args:
    predictions (list of str): The generated text from the RAG model.
    references (list of str): The reference ground truth text.

    Returns:
    dict: BLEU and ROUGE scores.
    """
    # BLEU score
    bleu_scores = []
    for prediction, reference in zip(predictions, references):
        reference_tokens = nltk.word_tokenize(reference.lower())
        prediction_tokens = nltk.word_tokenize(prediction.lower())

        # Use smoothing function for BLEU
        smoothie = SmoothingFunction().method1
        bleu_score = sentence_bleu([reference_tokens], prediction_tokens, smoothing_function=smoothie)
        bleu_scores.append(bleu_score)

    avg_bleu_score = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0

    # ROUGE score
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for prediction, reference in zip(predictions, references):
        scores = rouge_scorer_instance.score(reference, prediction)
        rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
        rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
        rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

    avg_rouge_scores = {
        'rouge1': sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1']),
        'rouge2': sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2']),
        'rougeL': sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL']),
    }

    # Final Results
    results = {
        'avg_bleu_score': avg_bleu_score,
        'avg_rouge1_score': avg_rouge_scores['rouge1'],
        'avg_rouge2_score': avg_rouge_scores['rouge2'],
        'avg_rougeL_score': avg_rouge_scores['rougeL'],
    }

    return results


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [24]:

# Example usage

predictions = [
     "Water is considered a macronutrient because it provides more calories than any other macronutrient in the form of heat. Water is composed of hydrogen and oxygen atoms, and its molecular structure is highly hydrated. It has a high specific heat capacity, meaning that it takes more energy to increase its temperature by a given amount than it takes to increase the temperature of any other material. Water is the primary component of all living organisms, including humans, as it provides the bulk of their energy and nutrients. It is essential for the maintenance of body temperature, muscle contractions, and digestion. Water also plays a crucial role in hydrating the body's organs, tissues, and cells, helping them to function properly. In summary, water is considered a macro-nutrient because it provides the energy and nutrients that are required for life.."

]
references = ["Water is considered a macronutrient because the body requires it in large amounts, even though it does not yield calories or provide energy.."

]

results = evaluate_rag_model(predictions, references)
print("Evaluation Results:")
print(f"Average BLEU Score: {results['avg_bleu_score']:.4f}")
print(f"Average ROUGE-1 Score: {results['avg_rouge1_score']:.4f}")
print(f"Average ROUGE-2 Score: {results['avg_rouge2_score']:.4f}")
print(f"Average ROUGE-L Score: {results['avg_rougeL_score']:.4f}")

Evaluation Results:
Average BLEU Score: 0.0371
Average ROUGE-1 Score: 0.1963
Average ROUGE-2 Score: 0.0745
Average ROUGE-L Score: 0.1472


In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import numpy as np

# Function to calculate Precision, Recall, F1 Score for text comparison
def calculate_metrics(y_true, y_pred):
    """
    Calculate Precision, Recall, and F1 Score between ground truth (y_true) and model-generated (y_pred) text.

    Args:
    - y_true: list of reference text (ground truth)
    - y_pred: list of generated text (model output)

    Returns:
    - precision: Precision score
    - recall: Recall score
    - f1: F1 Score
    """
    # Tokenize and convert texts to a binary representation (1 if word exists in the text, else 0)
    y_true_tokens = [set(true.split()) for true in y_true]
    y_pred_tokens = [set(pred.split()) for pred in y_pred]

    # Flatten the sets of tokens into individual word lists for comparison
    all_words = set([word for sublist in y_true_tokens + y_pred_tokens for word in sublist])

    # Create binary vectors for precision, recall, and F1 score
    y_true_binary = [[1 if word in tokens else 0 for word in all_words] for tokens in y_true_tokens]
    y_pred_binary = [[1 if word in tokens else 0 for word in all_words] for tokens in y_pred_tokens]

    # Convert to numpy arrays for use in scikit-learn metrics
    y_true_array = np.array(y_true_binary)
    y_pred_array = np.array(y_pred_binary)

    # Calculate Precision, Recall, and F1 score using sklearn
    precision = precision_score(y_true_array, y_pred_array, average='micro')
    recall = recall_score(y_true_array, y_pred_array, average='micro')
    f1 = f1_score(y_true_array, y_pred_array, average='micro')

    return precision, recall, f1

In [25]:

# Example ground truth (reference) text
y_true = [
    "Water is considered a macronutrient because the body requires it in large amounts, even though it does not yield calories or provide energy.."

]

# Example generated text (model output)
y_pred = [
    "Water is considered a macronutrient because it provides more calories than any other macronutrient in the form of heat. Water is composed of hydrogen and oxygen atoms, and its molecular structure is highly hydrated. It has a high specific heat capacity, meaning that it takes more energy to increase its temperature by a given amount than it takes to increase the temperature of any other material. Water is the primary component of all living organisms, including humans, as it provides the bulk of their energy and nutrients. It is essential for the maintenance of body temperature, muscle contractions, and digestion. Water also plays a crucial role in hydrating the body's organs, tissues, and cells, helping them to function properly. In summary, water is considered a macro-nutrient because it provides the energy and nutrients that are required for life."

]

precision, recall, f1 = calculate_metrics(y_true, y_pred)

# Calculate Precision, Recall, and F1 Score#
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Precision: 0.1294
Recall: 0.5000
F1 Score: 0.2056


In [26]:
!pip install bert-score
import bert_score

# Reference and generated sentences
reference = ["Water is considered a macronutrient because the body requires it in large amounts, even though it does not yield calories or provide energy.."]

generated = ["Water is considered a macronutrient because it provides more calories than any other macronutrient in the form of heat. Water is composed of hydrogen and oxygen atoms, and its molecular structure is highly hydrated. It has a high specific heat capacity, meaning that it takes more energy to increase its temperature by a given amount than it takes to increase the temperature of any other material. Water is the primary component of all living organisms, including humans, as it provides the bulk of their energy and nutrients. It is essential for the maintenance of body temperature, muscle contractions, and digestion. Water also plays a crucial role in hydrating the body's organs, tissues, and cells, helping them to function properly. In summary, water is considered a macro-nutrient because it provides the energy and nutrients that are required for life."]

# Compute BERTScore
P, R, F1 = bert_score.score(generated, reference, lang="en")

# Print Precision, Recall, F1-score
print(f"Precision: {P.mean():.4f}")
print(f"Recall: {R.mean():.4f}")
print(f"F1-score: {F1.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.8418
Recall: 0.8990
F1-score: 0.8695
